# Regresión Lasso: Teoría y Aplicación

Este notebook explora la regresión Lasso, con énfasis en su capacidad de seleccionar variables, teoría subyacente, implementación práctica y análisis del modelo resultante.

## 1. ¿Por qué usar Lasso?

- Ridge regulariza pero **no elimina** variables.
- En modelos con muchas variables, puede ser útil seleccionar un subconjunto relevante.
- Lasso usa penalización L1, lo que favorece soluciones **esparsas** (muchos coeficientes en cero).


## 2. Fundamento teórico
La regresión Lasso minimiza:
$$\text{min} \left\{ \| y - X\beta \|^2 + \lambda \|\beta\|_1 \right\}$$

- $\|\beta\|_1 = \sum_j |\beta_j|$ es la norma L1.
- $\lambda$ controla la penalización.
- Lasso puede poner algunos $\beta_j = 0$ si no aportan al modelo.

**Interpretación geométrica:** región de penalización en forma de diamante → intersecciones en ejes → coeficientes nulos.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from scipy.stats import shapiro

In [ ]:
# Preparar datos
X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Ajustar Lasso y OLS
lasso = Lasso(alpha=0.1, max_iter=10000).fit(X_train_scaled, y_train)
ols = LinearRegression().fit(X_train_scaled, y_train)

# Predicción
y_pred_lasso = lasso.predict(X_test_scaled)
y_pred_ols = ols.predict(X_test_scaled)

# Errores
print("Lasso:", mean_squared_error(y_test, y_pred_lasso), r2_score(y_test, y_pred_lasso))
print("OLS:", mean_squared_error(y_test, y_pred_ols), r2_score(y_test, y_pred_ols))

In [ ]:
# Coeficientes
print("Coeficientes Lasso:", lasso.coef_)
print("Coeficientes OLS:", ols.coef_)

In [ ]:
# Gráfico de coeficientes
plt.figure(figsize=(10, 5))
plt.plot(lasso.coef_, label='Lasso')
plt.plot(ols.coef_, label='OLS')
plt.axhline(0, color='gray', linestyle='--')
plt.legend()
plt.title("Coeficientes estimados por Lasso vs OLS")
plt.xlabel("Índice de variable")
plt.ylabel("Valor del coeficiente")
plt.grid(True)
plt.show()

## 3. Análisis de residuos y supuestos
Analizamos normalidad, homocedasticidad y comportamiento de errores.

In [ ]:
# Residuos de Lasso
residuals = y_test - y_pred_lasso

fig, ax = plt.subplots(1, 2, figsize=(12, 4))
sns.histplot(residuals, kde=True, ax=ax[0])
sm.qqplot(residuals, line='s', ax=ax[1])
ax[0].set_title("Histograma de residuos")
ax[1].set_title("Q-Q plot")
plt.tight_layout()
plt.show()

# Test de normalidad
stat, p = shapiro(residuals)
print("Shapiro-Wilk p-valor:", p)

## 4. Trayectorias de coeficientes con \( \lambda \)
Exploramos cómo los coeficientes se vuelven cero conforme aumenta la regularización.

In [ ]:
alphas = np.logspace(-4, 0.5, 100)
coefs = []
for a in alphas:
    lasso = Lasso(alpha=a, max_iter=10000).fit(X_train_scaled, y_train)
    coefs.append(lasso.coef_)

plt.figure(figsize=(10, 6))
plt.plot(alphas, coefs)
plt.xscale("log")
plt.xlabel("lambda")
plt.ylabel("Coeficientes")
plt.title("Trayectoria de coeficientes en Lasso")
plt.grid(True)
plt.show()

## 5. Conclusión y aplicaciones
- Lasso es útil para selección de variables y simplificación del modelo
- Es popular en modelos donde $p \gg n$
- Se utiliza en genética, procesamiento de señales, ciencia de datos y econometría
- Puede combinarse con Elastic Net para mejorar estabilidad cuando hay colinealidad